In [1]:
import numpy as np
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model

Using TensorFlow backend.


In [2]:
timesteps = 10
input_dim = 26
latent_dim = 100

In [3]:
inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(latent_dim)(inputs)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

In [34]:
sequence_autoencoder.compile(loss='binary_crossentropy', optimizer='adadelta')

In [12]:
from scipy.sparse import dok_matrix
from itertools import product

def generate_buckets(input_dim, indices):
    size, timesteps = indices.shape
    vec = np.zeros((size, timesteps, input_dim))
    for i, j in product(range(size), range(timesteps)):
        vec[i, j, indices[i, j]] = 1
    return vec

In [13]:
indices = np.random.randint(input_dim, size=(100, timesteps))
inputarr = generate_buckets(input_dim, indices)

In [24]:
def get_maxindices(vec):
    size, timestep, vecdim = vec.shape
    indices = np.zeros((size, timesteps))
    for i, j in product(range(size), range(timestep)):
        indices[i, j] = np.argmax(vec[i, j, :])
    return indices

In [35]:
sequence_autoencoder.fit(inputarr, inputarr, epochs=100)

Epoch 1/100
100/100 [==============================] - 1s - loss: 0.1071     
Epoch 2/100
100/100 [==============================] - 0s - loss: 0.1067     
Epoch 3/100
100/100 [==============================] - 0s - loss: 0.1064     
Epoch 4/100
100/100 [==============================] - 0s - loss: 0.1062     
Epoch 5/100
100/100 [==============================] - 0s - loss: 0.1065     
Epoch 6/100
100/100 [==============================] - 0s - loss: 0.1061     
Epoch 7/100
100/100 [==============================] - 0s - loss: 0.1060     
Epoch 8/100
100/100 [==============================] - 0s - loss: 0.1059     
Epoch 9/100
100/100 [==============================] - 0s - loss: 0.1058     
Epoch 10/100
100/100 [==============================] - 0s - loss: 0.1057     
Epoch 11/100
100/100 [==============================] - 0s - loss: 0.1057     
Epoch 12/100
100/100 [==============================] - 0s - loss: 0.1067     
Epoch 13/100
100/100 [==============================] - 0s - 

In [36]:
get_maxindices(inputarr)

array([[ 10.,   3.,   1.,  17.,   2.,   4.,  20.,  25.,   1.,   7.],
       [ 16.,  12.,   2.,  10.,  13.,  18.,   1.,  14.,  14.,  18.],
       [ 20.,  19.,  13.,   0.,   9.,   1.,  16.,  25.,  14.,   3.],
       [  0.,  15.,   8.,   4.,   3.,  23.,   0.,   6.,  21.,   0.],
       [  3.,  18.,  18.,   6.,  12.,  16.,  11.,  15.,   4.,   2.],
       [ 20.,  20.,  21.,  25.,   4.,  13.,  25.,   1.,   7.,  21.],
       [  3.,  19.,   2.,  21.,  15.,  20.,  25.,   1.,   1.,   3.],
       [ 22.,   1.,   9.,  14.,   6.,   9.,  10.,  12.,  16.,  25.],
       [  9.,   9.,  13.,  15.,  13.,  13.,  10.,   8.,   7.,  17.],
       [ 15.,  12.,   8.,   2.,  17.,  15.,   8.,   8.,  10.,  19.],
       [ 17.,  15.,  17.,   3.,  22.,   1.,  14.,   1.,  21.,  21.],
       [ 21.,  23.,  25.,   5.,  15.,  18.,  23.,  25.,  17.,   5.],
       [  7.,  25.,   5.,   6.,   7.,   8.,  23.,   2.,  10.,   8.],
       [ 14.,  16.,   8.,  16.,  25.,  11.,  15.,  24.,  12.,   7.],
       [ 13.,   2.,  11.,  13.,  2

In [27]:
encoder.predict(inputarr).shape

(100, 100)

In [37]:
get_maxindices(sequence_autoencoder.predict(inputarr))

array([[  3.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.],
       [ 16.,  16.,  14.,  14.,  14.,  14.,  14.,  14.,  14.,  14.],
       [ 13.,  13.,  13.,   1.,   9.,  25.,  25.,  25.,  25.,  25.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 18.,  18.,  18.,  18.,  11.,  12.,  12.,  12.,  12.,  12.],
       [ 21.,  21.,  21.,  21.,  25.,  25.,  25.,   7.,   1.,   1.],
       [  3.,  15.,  15.,   1.,   1.,   1.,   1.,   1.,   1.,  25.],
       [  1.,   9.,   9.,   9.,   9.,   9.,  25.,  25.,  25.,  25.],
       [ 13.,  13.,  13.,  13.,  13.,  13.,  13.,  13.,  13.,  13.],
       [ 15.,   8.,   8.,   8.,   8.,   8.,   8.,   8.,   8.,   8.],
       [  3.,  22.,  22.,   1.,   1.,  21.,  21.,  21.,  21.,  21.],
       [ 21.,  25.,  25.,  25.,  25.,  25.,  25.,  23.,   5.,   5.],
       [  5.,   7.,   7.,   7.,   7.,   7.,   7.,   2.,   2.,   2.],
       [ 16.,  16.,  16.,  16.,  16.,  16.,  16.,  15.,  15.,  15.],
       [ 13.,  13.,  13.,  13.,  1